## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# world-weather-analysis/weather_database/WeatherPy_Database.csv
city_data_df = pd.read_csv("/Users/zcvalis/Documents/Bootcamp/Module 6 - APIs/world-weather-analysis/weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Koran,BA,43.8103,18.5689,65.71,39,40,6.91,light rain
1,1,General Roca,AR,-39.0333,-67.5833,58.62,43,100,9.93,overcast clouds
2,2,Roald,NO,62.5833,6.1333,52.16,81,20,19.57,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.37,72,29,6.06,scattered clouds
4,4,Tasiilaq,GL,65.6145,-37.6368,37.56,73,98,11.74,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,76.37,72,29,6.06,scattered clouds
10,10,Doume,CM,4.2333,13.4500,87.31,48,89,4.00,light rain
15,15,Kavieng,PG,-2.5744,150.7967,81.77,75,8,4.41,clear sky
20,20,Matinhos,BR,-25.8175,-48.5428,75.49,73,81,5.01,broken clouds
22,22,Saint-Philippe,RE,-21.3585,55.7679,78.39,83,46,4.79,scattered clouds
24,24,Caucaia,BR,-3.7361,-38.6531,87.04,58,20,19.57,few clouds
26,26,Kudahuvadhoo,MV,2.6708,72.8944,86.13,75,88,17.54,overcast clouds
28,28,Puerto Ayora,EC,-0.7393,-90.3518,76.96,83,74,6.96,broken clouds
31,31,Pedernales,DO,18.0384,-71.7440,77.86,81,100,2.33,overcast clouds
32,32,Banjar,ID,-8.1900,114.9675,78.04,89,99,3.98,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                29
City                   29
Country                29
Lat                    29
Lng                    29
Max Temp               29
Humidity               29
Cloudiness             29
Wind Speed             29
Current Description    29
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.count()

City_ID                29
City                   29
Country                29
Lat                    29
Lng                    29
Max Temp               29
Humidity               29
Cloudiness             29
Wind Speed             29
Current Description    29
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,76.37,scattered clouds,-23.1203,-134.9692,
10,Doume,CM,87.31,light rain,4.2333,13.4500,
15,Kavieng,PG,81.77,clear sky,-2.5744,150.7967,
20,Matinhos,BR,75.49,broken clouds,-25.8175,-48.5428,
22,Saint-Philippe,RE,78.39,scattered clouds,-21.3585,55.7679,
24,Caucaia,BR,87.04,few clouds,-3.7361,-38.6531,
26,Kudahuvadhoo,MV,86.13,overcast clouds,2.6708,72.8944,
28,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
31,Pedernales,DO,77.86,overcast clouds,18.0384,-71.7440,
32,Banjar,ID,78.04,overcast clouds,-8.1900,114.9675,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
payload = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    payload["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=payload).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [11]:
clean_hotel_df.count()

City                   26
Country                26
Max Temp               26
Current Description    26
Lat                    26
Lng                    26
Hotel Name             26
dtype: int64

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,76.37,scattered clouds,-23.1203,-134.9692,People ThankYou
10,Doume,CM,87.31,light rain,4.2333,13.4500,Hotel Universel
15,Kavieng,PG,81.77,clear sky,-2.5744,150.7967,Nusa Island Retreat
20,Matinhos,BR,75.49,broken clouds,-25.8175,-48.5428,Hotel Pousada Girassol
22,Saint-Philippe,RE,78.39,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
24,Caucaia,BR,87.04,few clouds,-3.7361,-38.6531,M.O.S. Logística - Filial Fortaleza
26,Kudahuvadhoo,MV,86.13,overcast clouds,2.6708,72.8944,Niyama Private Islands Maldives
28,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
31,Pedernales,DO,77.86,overcast clouds,18.0384,-71.7440,Hotel Costa Marina
32,Banjar,ID,78.04,overcast clouds,-8.1900,114.9675,The Hamsa Resort


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))